In [1]:
from model_and_sampling import get_vllm_model

INFO 05-03 00:46:34 [__init__.py:239] Automatically detected platform cuda.


2025-05-03 00:46:36,066	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
from chat_prompts import generate_prompts_sql, generate_bd_list, generate_sql_gt_list

In [3]:
from process_and_save_answers import answers_process_pipeline

In [4]:
from score_sql import print_and_save

In [5]:
from process_and_save_answers import answers_process_pipeline

## Загружаем модель

In [6]:
model_config = {
    'model':"/data/home/vkropoti/models/Qwen3-8B",
    'tensor_parallel_size':2
}


In [7]:
llm, sampling_params = get_vllm_model(model=model_config['model'],
                                      tensor_parallel_size=model_config['tensor_parallel_size']
                                     )

INFO 05-02 22:52:23 [config.py:689] This model supports multiple tasks: {'reward', 'embed', 'generate', 'score', 'classify'}. Defaulting to 'generate'.
INFO 05-02 22:52:24 [config.py:1713] Defaulting to use mp for distributed inference
INFO 05-02 22:52:24 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 05-02 22:52:25 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='/data/home/vkropoti/models/Qwen3-8B', speculative_config=None, tokenizer='/data/home/vkropoti/models/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=5000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reaso

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  7.06it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.75it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  2.10it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  1.90it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.13it/s]
(VllmWorker rank=0 pid=1877039) 


(VllmWorker rank=1 pid=1877065) INFO 05-02 22:52:32 [loader.py:458] Loading weights took 1.89 seconds
(VllmWorker rank=0 pid=1877039) INFO 05-02 22:52:32 [loader.py:458] Loading weights took 1.91 seconds
(VllmWorker rank=1 pid=1877065) INFO 05-02 22:52:33 [gpu_model_runner.py:1291] Model loading took 7.6394 GiB and 2.015484 seconds
(VllmWorker rank=0 pid=1877039) INFO 05-02 22:52:33 [gpu_model_runner.py:1291] Model loading took 7.6394 GiB and 2.033307 seconds
(VllmWorker rank=1 pid=1877065) INFO 05-02 22:52:41 [backends.py:416] Using cache directory: /home/vkropoti/.cache/vllm/torch_compile_cache/1e8b5e7fc5/rank_1_0 for vLLM's torch.compile
(VllmWorker rank=1 pid=1877065) INFO 05-02 22:52:41 [backends.py:426] Dynamo bytecode transform time: 8.09 s
(VllmWorker rank=0 pid=1877039) INFO 05-02 22:52:41 [backends.py:416] Using cache directory: /home/vkropoti/.cache/vllm/torch_compile_cache/1e8b5e7fc5/rank_0_0 for vLLM's torch.compile
(VllmWorker rank=0 pid=1877039) INFO 05-02 22:52:41 [back

## Получаем данные

In [8]:
system_prompt = "You are helpfull assistant solving text to sql. Carefully answer the question and add <|im_end|> and the end of it."

In [9]:
prompts_config = {
    'path_dev_json':'/home/vkropoti/vllm/dev.json',
    'path_sql_dbs': "/data/home/vkropoti/sql_data/dev_databases/",
    'use_reasoning':False
}

In [10]:
%%time
batch_prompts = generate_prompts_sql(path_dev_json=prompts_config['path_dev_json'],
                                    path_sql_dbs=prompts_config['path_sql_dbs'],
                                     model_name=model_config['model'],
                                     system_prompt=system_prompt,
                                     use_reasoning = prompts_config['use_reasoning']
                                    )
bd_list = generate_bd_list(path_dev_json=prompts_config['path_dev_json'])
sql_gt_list = generate_sql_gt_list(path_dev_json=prompts_config['path_dev_json'])

CPU times: user 642 ms, sys: 146 ms, total: 788 ms
Wall time: 790 ms


## Генерация

In [11]:
%%time
outputs = llm.generate(batch_prompts,sampling_params=sampling_params)

Processed prompts: 100%|██████████| 1534/1534 [00:09<00:00, 157.90it/s, est. speed input: 163840.08 toks/s, output: 6987.00 toks/s]

CPU times: user 3.22 s, sys: 50.6 ms, total: 3.27 s
Wall time: 12.5 s


## Обработка и сохранение ответов

In [12]:
sql_predict = answers_process_pipeline(outputs)

extract_think количество ошибок:  0
extract_sql количество ошибок:  0


In [13]:
sql_predict[0]

"SELECT MAX(`Percent (%) Eligible Free (K-12)`) FROM frpm WHERE `County Name` = 'Alameda';"

## Подсчет метрик

In [14]:
path_sql_dbs = "/data/home/vkropoti/sql_data/dev_databases/"

In [15]:
%%time
print_and_save("Qwen3-8b",path_sql_dbs,bd_list,sql_gt_list,sql_predict)

Qwen3-8b Финальный результат EX: 0.452
Qwen3-8b gроцент запросов, которые успешно выполнились: 76.59%
Qwen3-8b Mini DEV Финальный результат EX: 0.414
Qwen3-8b процент запросов, которые успешно выполнились Mini DEV: 76.00%
CPU times: user 1min 58s, sys: 1min 40s, total: 3min 38s
Wall time: 1min 18s


In [15]:
%%time
print_and_save("Qwen3-8b",path_sql_dbs,bd_list,sql_gt_list,sql_predict)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f49c9df2d50>>
Traceback (most recent call last):
  File "/data/home/vkropoti/vllm/lib64/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

func_timeout.dafunc.FunctionTimedOut1840702833704239227: Function sqlite_operation (args=()) (kwargs={}) timed out after 5.000000 seconds.



Qwen3-8b Финальный результат EX: 0.452
Qwen3-8b gроцент запросов, которые успешно выполнились: 76.65%
Qwen3-8b Mini DEV Финальный результат EX: 0.414
Qwen3-8b процент запросов, которые успешно выполнились Mini DEV: 76.20%
CPU times: user 2min 2s, sys: 1min 35s, total: 3min 37s
Wall time: 1min 17s


In [23]:
# print_and_save??

In [4]:
from config import create_think_train_configs
from pipeline import pipeline, get_batch_example

ImportError: attempted relative import with no known parent package

In [5]:
def load_json(dir):
    with open(dir, "r") as j:
        contents = json.loads(j.read())
    return contents

In [7]:
f = load_json("/data/home/vkropoti/sql_data/train/train.json")

NameError: name 'json' is not defined

In [ ]:
model_config, prompts_config, save_config = create_think_train_configs(model,arr_seeds[0],arr_versions[0])
    llm = get_vllm_model(model=model_config['model'],
                                      tensor_parallel_size=model_config['tensor_parallel_size'],
                                          max_model_len=model_config['max_model_len'],
                                          gpu_memory_utilization=model_config['gpu_memory_utilization'],
                                          trust_remote_code=model_config['trust_remote_code'],
                                     )
    print("Модель успешно загружена")
    print("model_config",model_config)
    print()
    print("prompts_config",prompts_config)
    print()
    print("save_config",save_config)
    print()
    print("Модель успешно загружена")
    batch_example, max_tokens = get_batch_example(model_config, prompts_config)